In [2]:
# 1.Daily covid cases

import warnings
warnings.simplefilter(action='ignore')

import pandas as pd
import numpy as np
import datetime
# df=pd.read_excel(r'DataSource/airports.xlsx')
# df=df.drop(columns=['Unit'])
df=pd.read_excel(r'DataSource/flights.xlsx')
df=df.drop(columns=['更新时间','来源','频率','单位'])
df=df.dropna(subset=['指标名称'])
c1=df['指标名称'].str.contains('数据来源')
df=df[~c1]
df['指标名称']=df['指标名称'].str.replace('机场进出港航班:','')
df[['城市', '机场']] = df['指标名称'].str.split("/", expand = True)
df=df.drop(columns=['指标名称'])
cols=list(df.columns[-2:])+sorted(list(df.columns[:-2]),reverse=False)
df=df[cols]

df.to_hdf('./Processed_Data/airports.h5',key='raw')
df

,城市,机场,2019-01-01,2019-01-02,2019-01-03,2019-01-04,2019-01-05,2019-01-06,2019-01-07,2019-01-08,...,2022-12-30,2022-12-31,2023-01-01,2023-01-02,2023-01-03,2023-01-04,2023-01-05,2023-01-06,2023-01-07,2023-01-08
0,三明,沙县,NaN,10.0,NaN,11.0,NaN,10.0,10.0,NaN,...,11.0,NaN,10.0,10.0,8.0,8.0,7.0,8.0,6.0,6.0
1,无锡,硕放,134.0,141.0,142.0,145.0,144.0,145.0,147.0,148.0,...,93.0,99.0,98.0,104.0,110.0,111.0,115.0,115.0,121.0,132.0
2,宜昌,三峡,45.0,52.0,53.0,56.0,56.0,56.0,57.0,58.0,...,28.0,30.0,32.0,36.0,38.0,40.0,44.0,44.0,47.0,50.0
3,广州,白云,1261.0,1286.0,1285.0,1292.0,1292.0,1296.0,1294.0,1298.0,...,527.0,555.0,557.0,587.0,609.0,627.0,641.0,649.0,676.0,732.0
4,抚远,东极,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,...,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
228,义乌,None,35.0,37.0,35.0,36.0,37.0,37.0,36.0,38.0,...,26.0,27.0,26.0,26.0,27.0,28.0,30.0,31.0,32.0,36.0
229,呼和浩特,白塔,206.0,223.0,224.0,230.0,229.0,232.0,233.0,234.0,...,99.0,101.0,103.0,105.0,109.0,111.0,114.0,118.0,125.0,135.0
230,北京,大兴,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,346.0,366.0,370.0,394.0,411.0,421.0,431.0,436.0,454.0,487.0
231,鞍山,腾鳌,6.0,8.0,6.0,8.0,6.0,8.0,8.0,8.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
ap=pd.read_hdf('./Processed_Data/airports.h5',key='raw')
ap=ap.groupby(by='城市').sum().reset_index()
lastcol=ap.columns[-1]
ap=ap.sort_values(by=lastcol,ascending=False).head(50)
ap=pd.melt(ap,id_vars=['城市'],var_name='date',value_name='air_traffic')
ap=ap[ap['air_traffic']>0]

ap['date'] = pd.to_datetime(ap['date']).dt.date
ap=ap.sort_values(by=['城市','date'],ascending=True)
ap['date']=ap['date'].astype('str')

def getdt(d):
    return str(d)[-5:]
ap['dt']=ap['date'].apply(getdt)

ap['ap_last7d_mean']=ap.groupby('城市')['air_traffic'].transform(lambda x: x.rolling(7, 1).mean())
ap['ap_WoW']=ap['ap_last7d_mean']/ap.groupby('城市')['ap_last7d_mean'].transform(lambda x: x.shift(periods=7)) -1

c1=ap['date'].str.contains('2019')
ap_19=ap[c1]
ap_19=ap_19[['城市', 'dt','ap_last7d_mean']]
ap_19=ap_19.rename(columns={'ap_last7d_mean':'ap_19_rolling_mean'})

c2=ap['date'].str.contains('2022|2023')
ap_22=ap[c2]
ap_22=ap_22.merge(ap_19,how='left',on=['城市','dt'])

ap_22['ap_status']=ap_22['ap_last7d_mean']/ap_22['ap_19_rolling_mean']

ap_22.tail(3)

,城市,date,air_traffic,dt,ap_last7d_mean,ap_WoW,ap_19_rolling_mean,ap_status
18388,青岛,2023-01-06,226.0,01-06,213.285714,0.254622,460.500000,0.463161
18389,青岛,2023-01-07,234.0,01-07,217.571429,0.238211,461.428571,0.471517
18390,青岛,2023-01-08,257.0,01-08,225.857143,0.244882,464.000000,0.486761


In [4]:
# TY 机场 vs. 2019
%load_ext autoreload
%autoreload 2
from colored_list import dafeng_city,jidui_city,riqi
riqi='2023-01-08'

df_covid=pd.read_hdf('Processed_Data/Baidu_covid_progress.h5',key='raw')
o=ap_22.merge(df_covid,on=['城市'],how='left')

o=o[o['ap_status']<1.5][o['date']==riqi][['城市','progress_2','ap_status']] 

o['size']=np.nan
o['group']='其他'

c1=o['城市'].isin(dafeng_city)
o.loc[c1,'group']='本波疫情基本结束'

o=o.sort_values(by='group',ascending=False)
o.to_clipboard(index=False,header=None)

d=ap_22[ap_22['date']==riqi][['城市','date','ap_status']].reset_index(drop=True).rename(columns={'ap_status':'airports'})
d['date']=pd.to_datetime(d['date'])
d.to_hdf('Processed_Data/city_compare.h5',key='airports')

print(riqi)
print(o)

2023-01-08
         城市  progress_2  ap_status  size     group
18390    青岛         1.0   0.486761   NaN  本波疫情基本结束
6247     天津         0.0   0.443491   NaN  本波疫情基本结束
12871    深圳         0.0   0.762760   NaN  本波疫情基本结束
14356   石家庄         0.0   0.456970   NaN  本波疫情基本结束
12129    济南         0.0   0.636735   NaN  本波疫情基本结束
11032    沈阳         1.0   0.660141   NaN  本波疫情基本结束
10661    武汉         0.0   0.639920   NaN  本波疫情基本结束
10290    杭州         0.0   0.675381   NaN  本波疫情基本结束
9919     昆明         0.0   0.640448   NaN  本波疫情基本结束
9548     无锡         1.0   0.798419   NaN  本波疫情基本结束
8434     成都         0.0   1.067454   NaN  本波疫情基本结束
8061     徐州         0.0   0.865014   NaN  本波疫情基本结束
7731     广州         0.0   0.499945   NaN  本波疫情基本结束
7360     常州         0.0   0.622951   NaN  本波疫情基本结束
6989     宁波         0.0   0.703218   NaN  本波疫情基本结束
13985    珠海         0.0   0.637812   NaN  本波疫情基本结束
6618     太原         1.0   0.496493   NaN  本波疫情基本结束
15830    西安         0.0   0.514626   NaN  本波疫情基本结束
16200    贵阳         

In [5]:
# TY 机场数据 WoW
# %load_ext autoreload
# %autoreload 2
# from colored_list import dafeng_city,jidui_city,riqi

df_covid=pd.read_hdf('Processed_Data/Baidu_covid_progress.h5',key='raw')
o=ap_22.merge(df_covid,on=['城市'],how='left')

o=o[o['date']==riqi][['城市','progress_2','ap_WoW']] 

o['size']=np.nan
o['group']='其他'

c1=o['城市'].isin(dafeng_city)
o.loc[c1,'group']='本波疫情基本结束'

o=o.sort_values(by='group',ascending=False)

o.to_clipboard(index=False,header=None)
print(riqi)

2023-01-08


In [9]:
# TY 机场数据 WoW vs. GDP
# %load_ext autoreload
# %autoreload 2
# from colored_list import dafeng_city,jidui_city,riqi

city_popu=pd.read_hdf('Processed_Data/citypopu.h5',key='raw2')
o=ap_22.merge(city_popu,on=['城市'],how='left')

o=o[o['date']==riqi][['城市','GDP','ap_WoW']] 

o['size']=np.nan
o['group']='其他'

c1=o['城市'].isin(dafeng_city)
o.loc[c1,'group']='本波疫情基本结束'

o=o.sort_values(by='group',ascending=False)

o.to_clipboard(index=False,header=None)
print(riqi)

2023-01-08


In [10]:
# TY 机场数据 as of 19 vs. GDP
# %load_ext autoreload
# %autoreload 2
# from colored_list import dafeng_city,jidui_city,riqi

city_popu=pd.read_hdf('Processed_Data/citypopu.h5',key='raw2')
o=ap_22.merge(city_popu,on=['城市'],how='left')

o=o[o['date']==riqi][o['ap_status']<1.5][['城市','GDP','ap_status']] 

o['size']=np.nan
o['group']='其他'

c1=o['城市'].isin(dafeng_city)
o.loc[c1,'group']='本波疫情基本结束'

o=o.sort_values(by='group',ascending=False)

o.to_clipboard(index=False,header=None)
print(riqi)

2023-01-08


In [24]:
# d1=pd.read_clipboard()
# d1.to_hdf('Processed_Data/tablehead.h5','airports')
d1=pd.read_hdf('Processed_Data/tablehead.h5','airports')

apo=pd.read_hdf('./Processed_Data/airports.h5',key='raw')
cols=[i for i in apo.columns if i >'2023-01-04']

d1.merge(apo[cols],on=['城市','机场'],how='left').drop(columns=['城市','机场']).to_clipboard(index=False)